In [546]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

In [547]:
def convolve2d(matrix, kernel, type='valid'):
    """
    isto kao cross_correlate, samo se kernel rotira 180
    """
    # Get the dimensions of the input matrix and kernel
    m, n = matrix.shape
    km, kn = kernel.shape

    if type == 'valid':
    
        # Calculate the dimensions of the output matrix
        output_dim_m = m - km + 1
        output_dim_n = n - kn + 1
        output = np.zeros((output_dim_m, output_dim_n))
        
        # Flip the kernel for convolution
        kernel_flipped = np.rot90(kernel, 2) # or kernel_flipped = np.flipud(np.fliplr(kernel))
        
        # Perform the convolution
        for i in range(output_dim_m):
            for j in range(output_dim_n):
                # Element-wise multiplication and summation
                region = matrix[i:i+km, j:j+kn]
                output[i, j] = np.sum(region * kernel_flipped)
        
        return output
    
    elif type == 'full':

        output_dim_m = m + km - 1
        output_dim_n = n + kn - 1
        output = np.zeros((output_dim_m, output_dim_n))

        kernel_flipped = np.rot90(kernel, 2)

        padded_matrix = np.pad(matrix, ((km - 1, km - 1), (kn - 1, kn - 1)), mode='constant')

        for i in range(output_dim_m):
            for j in range(output_dim_n):
                region = padded_matrix[i:i+km, j:j+kn]
                output[i, j] = np.sum(region * kernel_flipped)

        return output

def cross_correlate2d(matrix, kernel, type='valid'): 
    """
    OVO RADI

    dimenzija rezultata = dim_input - dim_kernel + 1
    Y = I - K + 1

    slidea kernel po regijama matrice velicine kernela, mnoze se elementi i zbrajaju

    valid - krece se u granicama matrice, od ruba do ruba, dimenzija je Y = I - K + 1

    full - izlazi van granica matrice, treba paddati matricu s nulama, Y = I + K - 1
    """
    # Get the dimensions of the input matrix and kernel
    m, n = matrix.shape
    km, kn = kernel.shape

    if type == 'valid':
        
        # Calculate the dimensions of the output matrix
        output_dim_m = m - km + 1
        output_dim_n = n - kn + 1
        output = np.zeros((output_dim_m, output_dim_n))
        
        # Perform the cross-correlation
        for i in range(output_dim_m):
            for j in range(output_dim_n):
                # Element-wise multiplication and summation
                region = matrix[i:i+km, j:j+kn]
                output[i, j] = np.sum(region * kernel)
        
        return output
    
    elif type == 'full':
        
        # Calculate the dimensions of the output matrix
        output_dim_m = m + km - 1
        output_dim_n = n + kn - 1
        output = np.zeros((output_dim_m, output_dim_n))
        
        # Pad the input matrix with zeros
        padded_matrix = np.pad(matrix, ((km-1, km-1), (kn-1, kn-1)), mode='constant')

        # Perform the cross-correlation
        for i in range(output_dim_m):
            for j in range(output_dim_n):
                # Element-wise multiplication and summation
                region = padded_matrix[i:i+km, j:j+kn]
                output[i, j] = np.sum(region * kernel)
        
        return output
        

# Cross Correlate

In [548]:
a = np.array([[1, 6, 2],
              [5, 3, 1],
              [7, 0 ,4]])

kernel = np.array([[1, 2],
                   [-1, 0]])

c = cross_correlate2d(a, kernel)
print(c)

c = signal.correlate2d(a, kernel, mode='valid')
print(c)

[[8. 7.]
 [4. 5.]]
[[8 7]
 [4 5]]


# Cross Correlate full

In [549]:
a = np.array([[1, 6, 2],
              [5, 3, 1],
              [7, 0 ,4]])

kernel = np.array([[1, 2],
                   [-1, 0]])

c = cross_correlate2d(a, kernel, type='full')
print(c)

c = signal.correlate2d(a, kernel)
print(c)

[[ 0. -1. -6. -2.]
 [ 2.  8.  7.  1.]
 [10.  4.  5. -3.]
 [14.  7.  8.  4.]]
[[ 0 -1 -6 -2]
 [ 2  8  7  1]
 [10  4  5 -3]
 [14  7  8  4]]


# Convolve

In [550]:
a = np.array([[1, 6, 2],
              [5, 3, 1],
              [7, 0 ,4]])

kernel = np.array([[1, 2],
                   [-1, 0]])

c = convolve2d(a, kernel)
print(c)

c = signal.convolve2d(a, kernel, mode='valid')
print(c)

[[ 7.  5.]
 [11.  3.]]
[[ 7  5]
 [11  3]]


# Convolve full

In [551]:
a = np.array([[1, 6, 2],
              [5, 3, 1],
              [7, 0 ,4]])

kernel = np.array([[1, 2],
                   [-1, 0]])

c = convolve2d(a, kernel, type='full')
print(c)

c = signal.convolve2d(a, kernel)
print(c)

[[ 1.  8. 14.  4.]
 [ 4.  7.  5.  2.]
 [ 2. 11.  3.  8.]
 [-7.  0. -4.  0.]]
[[ 1  8 14  4]
 [ 4  7  5  2]
 [ 2 11  3  8]
 [-7  0 -4  0]]


In [552]:
class ConvolutionalLayer:

    def __init__(self, input_shape, kernel_size, depth) -> None:
        input_depth, input_height, input_width = input_shape
        self.depth = depth
        self.input_shape = input_shape
        self.input_depth = input_depth
        self.output_shape = (depth, input_height - kernel_size + 1, input_width - kernel_size + 1)
        self.kernels_shape = (depth, input_depth, kernel_size, kernel_size)
        self.kernels = np.random.randn(*self.kernels_shape)
        self.biases = np.random.randn(*self.output_shape)

    def forward(self, inputs):
        n_samples = inputs.shape[0]
        self.inputs = inputs
        self.output = np.zeros((n_samples, *self.output_shape))

        for i in range(self.depth):
            for j in range(self.input_depth):
                for k in range(n_samples):
                    self.output[k, i] += signal.correlate2d(self.inputs[k, j], self.kernels[i, j], mode="valid")
                self.output[i] += self.biases[i]

    def backward(self, delta):
        self.dkernels = np.zeros(self.kernels.shape)
        self.dbiases = np.zeros(self.biases.shape)
        self.dinputs = np.zeros(self.inputs.shape)

        for i in range(self.depth):
            for j in range(self.input_depth):
                for k in range(len(self.inputs)):
                    self.dkernels[i, j] += signal.correlate2d(self.inputs[k, j], delta[k, j], "valid")
                    self.dbiases[i] += delta[k, j]
                    self.dinputs[k, j] += signal.convolve2d(delta[k, j], self.kernels[i, j], "full")
             

In [553]:
class ReshapeLayer:

    def __init__(self, input_shape, output_shape) -> None:
        self.input_shape = input_shape
        self.output_shape = output_shape

    def forward(self, inputs):
        self.output = np.reshape(inputs, self.output_shape)

    def backward(self, delta):
        self.dinputs = np.reshape(delta, self.input_shape)

In [554]:
from keras.datasets import mnist

def preprocess_data(x, y, limit):
    zero_index = np.where(y == 0)[0][:limit]
    one_index = np.where(y == 1)[0][:limit]
    all_indices = np.hstack((zero_index, one_index))
    all_indices = np.random.permutation(all_indices)
    x, y = x[all_indices], y[all_indices]
    x = x.reshape(len(x), 1, 28, 28)
    x = x.astype("float32") / 255
    return x, y


In [555]:
# load MNIST from server, limit to 100 images per class since we're not training on GPU
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, y_train = preprocess_data(x_train, y_train, 100)
x_test, y_test = preprocess_data(x_test, y_test, 100)
n_samples = x_train.shape[0]

In [556]:
from DLFS import Sigmoid, DenseLayer, Model, BCE_Loss, Optimizer_SGD

layers = [ConvolutionalLayer((1, 28, 28), 3, 5),
          Sigmoid(),
          ReshapeLayer((n_samples, 5, 26, 26), (n_samples , 5 * 26 * 26)),
          DenseLayer(5 * 26 * 26, 100),
          Sigmoid(),
          DenseLayer(100, 1),
          Sigmoid()]

model = Model(layers=layers, loss_function=BCE_Loss(), optimizer=Optimizer_SGD())

model.train(x_train, y_train.reshape(-1, 1), print_every=20, iterations=150)

Loss: 0.67557943624563
Loss: 0.47256092801409544
Loss: 0.08969985390686024
Loss: 0.050047732318209304
Loss: 0.0342821759744453
Loss: 0.026043209568767257
Loss: 0.020896348974414322
Loss: 0.01735034872717643


In [557]:
y_pred = model.predict(x_test)
print(f'acc: {np.mean(np.round(y_pred) == y_test.reshape(-1, 1))}')

acc: 1.0
